<a href="https://colab.research.google.com/github/Incognito005/KNN_DuDoan_KQBD/blob/main/KNN_D%E1%BB%B1_%C4%91o%C3%A1n_KQBD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import warnings

warnings.filterwarnings('ignore')

In [68]:
# CÁC HÀM FEATURE ENGINEERING

In [69]:
def preprocess_data(df):
    """Tiền xử lý cơ bản: Sắp xếp theo ngày, tạo cột chỉ báo thắng/thua/hòa, và mã hóa Target."""

    # Chuyển đổi MatchDate sang datetime và sắp xếp theo thứ tự thời gian
    df['MatchDate'] = pd.to_datetime(df['MatchDate'], dayfirst=False)
    df = df.sort_values(by='MatchDate').reset_index(drop=True)

    # Tạo các cột chỉ báo Thắng/Hòa/Thua (dạng 0/1)
    df['HomeWin'] = (df['FullTimeResult'] == 'H').astype(int)
    df['HomeDraw'] = (df['FullTimeResult'] == 'D').astype(int)
    df['HomeLoss'] = (df['FullTimeResult'] == 'A').astype(int)
    df['AwayWin'] = (df['FullTimeResult'] == 'A').astype(int)
    df['AwayDraw'] = (df['FullTimeResult'] == 'D').astype(int)
    df['AwayLoss'] = (df['FullTimeResult'] == 'H').astype(int)

    # Mã hóa biến mục tiêu (Label): H -> 1, D -> 0, A -> 2
    label_mapping = {'H': 1, 'D': 0, 'A': 2}
    df['Target'] = df['FullTimeResult'].map(label_mapping)

    return df

In [70]:
def calculate_rolling_features(df, window=5):
    """Tính toán các features lịch sử lăn (rolling features) 5 trận gần nhất cho mỗi đội."""

    # Tạo DataFrame chung cho Hiệu suất của TẤT CẢ các trận đấu từ góc nhìn của MỘT ĐỘI
    team_home = df[['MatchDate', 'HomeTeam', 'HomeWin', 'HomeDraw', 'FullTimeHomeGoals', 'FullTimeAwayGoals', 'HomeShotsOnTarget', 'AwayShotsOnTarget']].copy()
    team_home.columns = ['MatchDate', 'Team', 'Win', 'Draw', 'GoalsFor', 'GoalsAgainst', 'ShotsOnTargetFor', 'ShotsOnTargetAgainst']
    team_home['IsHome'] = 1

    team_away = df[['MatchDate', 'AwayTeam', 'AwayWin', 'AwayDraw', 'FullTimeAwayGoals', 'FullTimeHomeGoals', 'AwayShotsOnTarget', 'HomeShotsOnTarget']].copy()
    team_away.columns = ['MatchDate', 'Team', 'Win', 'Draw', 'GoalsFor', 'GoalsAgainst', 'ShotsOnTargetFor', 'ShotsOnTargetAgainst']
    team_away['IsHome'] = 0

    team_performance = pd.concat([team_home, team_away], ignore_index=True)
    team_performance = team_performance.sort_values(by=['MatchDate', 'Team'])

    def calculate_stats(group):
        # shift(1) RẤT QUAN TRỌNG: Đảm bảo không sử dụng kết quả của trận đấu hiện tại
        shifted_group = group.sort_values(by='MatchDate').shift(1)
        stats = shifted_group.rolling(window=window, min_periods=1)

        # 1. Tính toán các chỉ số chung (Rolling 5)
        group[f'WinRate{window}'] = stats['Win'].mean()
        group[f'DrawRate{window}'] = stats['Draw'].mean()
        group[f'AvgGoalsFor{window}'] = stats['GoalsFor'].mean()
        group[f'AvgGoalsAgainst{window}'] = stats['GoalsAgainst'].mean()
        group[f'AvgShotsOnTargetFor{window}'] = stats['ShotsOnTargetFor'].mean()
        group[f'AvgShotsOnTargetAgainst{window}'] = stats['ShotsOnTargetAgainst'].mean()

        # 2. Tỷ lệ thắng Sân Nhà/Sân Khách (chỉ xét Home/Away Games trước đó)
        shifted_group['HomeGameWin'] = shifted_group['Win'].where(shifted_group['IsHome'] == 1)
        shifted_group['AwayGameWin'] = shifted_group['Win'].where(shifted_group['IsHome'] == 0)

        full_home_rate = shifted_group['HomeGameWin'].rolling(window=window, min_periods=1).mean()
        full_away_rate = shifted_group['AwayGameWin'].rolling(window=window, min_periods=1).mean()

        # Gán kết quả rolling rate vào vị trí tương ứng
        group[f'HomeHomeWinRate{window}'] = np.where(group['IsHome'] == 1, full_home_rate, np.nan)
        group[f'AwayAwayWinRate{window}'] = np.where(group['IsHome'] == 0, full_away_rate, np.nan)

        return group

    # Áp dụng hàm tính toán cho mỗi đội
    team_performance = team_performance.groupby('Team', group_keys=False).apply(calculate_stats).reset_index(drop=True)

    # Lấy các cột tính toán được và Merge trở lại DataFrame gốc (df)
    cols_to_merge = [col for col in team_performance.columns if any(s in col for s in ['Rate', 'Avg'])]
    home_features = team_performance[team_performance['IsHome'] == 1].rename(
        columns={c: 'Home_' + c for c in cols_to_merge}
    )[['MatchDate', 'Team'] + ['Home_' + c for c in cols_to_merge]]
    df = pd.merge(df, home_features.rename(columns={'Team': 'HomeTeam'}), on=['MatchDate', 'HomeTeam'], how='left')

    away_features = team_performance[team_performance['IsHome'] == 0].rename(
        columns={c: 'Away_' + c for c in cols_to_merge}
    )[['MatchDate', 'Team'] + ['Away_' + c for c in cols_to_merge]]
    df = pd.merge(df, away_features.rename(columns={'Team': 'AwayTeam'}), on=['MatchDate', 'AwayTeam'], how='left')

    # Đổi tên cột cho khớp với yêu cầu
    df.rename(columns={
        'Home_WinRate5': 'HomeWinRate5', 'Away_WinRate5': 'AwayWinRate5',
        'Home_DrawRate5': 'HomeDrawRate5', 'Away_DrawRate5': 'AwayDrawRate5',
        'Home_AvgGoalsFor5': 'HomeAvgGoals5', 'Away_AvgGoalsFor5': 'AwayAvgGoals5',
        'Home_AvgGoalsAgainst5': 'HomeAvgGoalsConceded5', 'Away_AvgGoalsAgainst5': 'AwayAvgGoalsConceded5',
        'Home_AvgShotsOnTargetFor5': 'HomeAvgShotOnTarget5', 'Away_AvgShotsOnTargetFor5': 'AwayAvgShotOnTarget5',
        'Home_AvgShotsOnTargetAgainst5': 'HomeAvgShotsConceded5', 'Away_AvgShotsOnTargetAgainst5': 'AwayAvgShotsConceded5',
        'Home_HomeHomeWinRate5': 'HomeHomeWinRate5',
        'Away_AwayAwayWinRate5': 'AwayAwayWinRate5',
    }, inplace=True)
    return df

In [71]:
def calculate_h2h_features(df, window=5):
    """Tính toán các features Đối đầu (Head-to-Head) 5 trận gần nhất."""
    df = df.copy()
    df = df.sort_values(by='MatchDate').reset_index(drop=True)

    df['Head2Head_HomeWinRate5'] = np.nan
    df['Head2Head_DrawRate5'] = np.nan
    df['Head2Head_AwayWinRate5'] = np.nan

    for idx, row in df.iterrows():
        team1, team2 = row['HomeTeam'], row['AwayTeam']
        match_date = row['MatchDate']

        # Lọc các trận đối đầu TRƯỚC trận đấu hiện tại (5 trận gần nhất)
        past_matches = df[
            (((df['HomeTeam'] == team1) & (df['AwayTeam'] == team2)) |
             ((df['HomeTeam'] == team2) & (df['AwayTeam'] == team1))) &
            (df['MatchDate'] < match_date)
        ].sort_values(by='MatchDate', ascending=False).head(window)

        total = len(past_matches)

        if total == 0:
            continue

        home_wins = draws = away_wins = 0

        for _, match in past_matches.iterrows():
            if match['FullTimeResult'] == 'D':
                draws += 1
            # team1 (đội nhà của trận hiện tại) thắng
            elif (match['HomeTeam'] == team1 and match['FullTimeResult'] == 'H') or \
                 (match['AwayTeam'] == team1 and match['FullTimeResult'] == 'A'):
                home_wins += 1
            # team2 (đội khách của trận hiện tại) thắng
            else:
                away_wins += 1

        df.loc[idx, 'Head2Head_HomeWinRate5'] = home_wins / total
        df.loc[idx, 'Head2Head_DrawRate5'] = draws / total
        df.loc[idx, 'Head2Head_AwayWinRate5'] = away_wins / total

    return df

In [72]:
def calculate_league_averages(df):
    """Tính trung bình giải đấu để điền vào các giá trị bị thiếu (NaN)."""
    league_avg = {
        'win_rate': 0.33,
        'draw_rate': 0.27,
        'goals': df[['FullTimeHomeGoals', 'FullTimeAwayGoals']].mean().mean(),
        'shots_on_target': df[['HomeShotsOnTarget', 'AwayShotsOnTarget']].mean().mean()
    }
    return league_avg

In [73]:
# TẢI DỮ LIỆU & FEATURE ENGINEERING

In [74]:
# Tải dữ liệu
print("\nĐang tải dữ liệu...")
df = pd.read_csv('/content/drive/My Drive/Data_ML/epl_24-25.csv')
print(f"Đã tải thành công {len(df)} trận đấu")


Đang tải dữ liệu...
Đã tải thành công 730 trận đấu


In [75]:
# Feature Engineering
print("\nĐang tạo Feature Engineering...")
df = preprocess_data(df)
df = calculate_rolling_features(df, window=5)
df = calculate_h2h_features(df, window=5)
print("Đã hoàn thành tạo Features")


Đang tạo Feature Engineering...
Đã hoàn thành tạo Features


In [76]:
# Định nghĩa Features
feature_cols = [
    'HomeWinRate5', 'AwayWinRate5', 'HomeDrawRate5', 'AwayDrawRate5',
    'HomeAvgGoals5', 'AwayAvgGoals5', 'HomeAvgGoalsConceded5', 'AwayAvgGoalsConceded5',
    'HomeHomeWinRate5', 'AwayAwayWinRate5',
    'HomeAvgShotOnTarget5', 'AwayAvgShotOnTarget5',
    'HomeAvgShotsConceded5', 'AwayAvgShotsConceded5',
    'Head2Head_HomeWinRate5', 'Head2Head_DrawRate5', 'Head2Head_AwayWinRate5'
]

X = df[feature_cols]
y = df['Target']

In [77]:
# Xử lý giá trị thiếu (NaN) bằng trung bình giải đấu (Impute)
league_avg = calculate_league_averages(df)
impute_values = {
    'HomeWinRate5': league_avg['win_rate'], 'AwayWinRate5': league_avg['win_rate'],
    'HomeDrawRate5': league_avg['draw_rate'], 'AwayDrawRate5': league_avg['draw_rate'],
    'HomeAvgGoals5': league_avg['goals'], 'AwayAvgGoals5': league_avg['goals'],
    'HomeAvgGoalsConceded5': league_avg['goals'], 'AwayAvgGoalsConceded5': league_avg['goals'],
    'HomeHomeWinRate5': league_avg['win_rate'], 'AwayAwayWinRate5': league_avg['win_rate'],
    'HomeAvgShotOnTarget5': league_avg['shots_on_target'], 'AwayAvgShotOnTarget5': league_avg['shots_on_target'],
    'HomeAvgShotsConceded5': league_avg['shots_on_target'], 'AwayAvgShotsConceded5': league_avg['shots_on_target'],
    'Head2Head_HomeWinRate5': league_avg['win_rate'],
    'Head2Head_DrawRate5': league_avg['draw_rate'],
    'Head2Head_AwayWinRate5': league_avg['win_rate']
}
X = X.fillna(impute_values)

In [78]:
# Chia dữ liệu theo chuỗi thời gian (Time-series split 80/20)
split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"\nKích thước tập huấn luyện (Train): {len(X_train)}, Kích thước tập kiểm tra (Test): {len(X_test)}")


Kích thước tập huấn luyện (Train): 584, Kích thước tập kiểm tra (Test): 146


In [79]:
# Chuẩn hóa dữ liệu (Scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print("Đã hoàn thành chuẩn hóa dữ liệu.")

Đã hoàn thành chuẩn hóa dữ liệu.


In [80]:
# TÌM K TỐI ƯU & HUẤN LUYỆN MÔ HÌNH

In [81]:
print("\nĐang chạy Grid Search tìm K tối ưu cho KNN...")
param_grid = {'n_neighbors': [3, 5, 7, 9, 11, 13, 15]}
# CV=5: Cross-validation 5 lần trên tập huấn luyện
knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
knn_grid.fit(X_train_scaled, y_train)

best_k = knn_grid.best_params_['n_neighbors']
print(f"K tối ưu tìm được = {best_k} (Độ chính xác CV: {knn_grid.best_score_:.4f})")


Đang chạy Grid Search tìm K tối ưu cho KNN...
K tối ưu tìm được = 15 (Độ chính xác CV: 0.4674)


In [82]:
# HUẤN LUYỆN MÔ HÌNH KNN TỐI ƯU VÀ ĐÁNH GIÁ CUỐI CÙNG

best_k = knn_grid.best_params_['n_neighbors']
print(f"\nĐang huấn luyện KNN với K tối ưu = {best_k}...")

knn_final = KNeighborsClassifier(n_neighbors=best_k)
knn_final.fit(X_train_scaled, y_train)

# Dự đoán và đánh giá
y_pred_knn = knn_final.predict(X_test_scaled)
acc_knn = accuracy_score(y_test, y_pred_knn)

print(f"\nKẾT QUẢ ĐÁNH GIÁ MÔ HÌNH KNN (K={best_k})")
print(f"\nĐộ chính xác Test Set: {acc_knn:.4f}")
print("\nBáo cáo phân loại:")
print(classification_report(y_test, y_pred_knn,
                            target_names=['Draw (0)', 'HomeWin (1)', 'AwayWin (2)']))

# Ma trận nhầm lẫn
cm = confusion_matrix(y_test, y_pred_knn)
print("\nMa trận nhầm lẫn:")
print("                Dự đoán (Predicted)")
print("               Hòa  Chủ nhà  Khách")
print(f"Thực tế Hòa    {cm[0,0]:3d}  {cm[0,1]:5d}  {cm[0,2]:5d}")
print(f"Thực tế Chủ nhà{cm[1,0]:3d}  {cm[1,1]:5d}  {cm[1,2]:5d}")
print(f"Thực tế Khách  {cm[2,0]:3d}  {cm[2,1]:5d}  {cm[2,2]:5d}")


print("\nHoàn thành phân tích KNN!")



Đang huấn luyện KNN với K tối ưu = 15...

KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH KNN (K=15)

Độ chính xác Test Set: 0.5342

Báo cáo phân loại:
              precision    recall  f1-score   support

    Draw (0)       0.27      0.21      0.24        29
 HomeWin (1)       0.59      0.75      0.66        64
 AwayWin (2)       0.57      0.45      0.51        53

    accuracy                           0.53       146
   macro avg       0.48      0.47      0.47       146
weighted avg       0.52      0.53      0.52       146


Ma trận nhầm lẫn:
                Dự đoán (Predicted)
               Hòa  Chủ nhà  Khách
Thực tế Hòa      6     14      9
Thực tế Chủ nhà  7     48      9
Thực tế Khách    9     20     24

Hoàn thành phân tích KNN!
